In [1]:
import tensorflow as tf
import numpy as np
import pylab as pl
import pandas as pd

%matplotlib inline

In [2]:
from sklearn.datasets import california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [3]:
california=california_housing.fetch_california_housing()

In [4]:
x=scale(california['data'])
y=california['target']

df=pd.DataFrame(x,columns=california['feature_names'])

In [5]:
df2=pd.read_csv("/data/california/housing.csv")

In [6]:
print(y.mean())
df2.median_house_value.mean()

2.06855816909


206855.81690891474

In [7]:
# Splitting into train and test sets

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3)

# Random Forest regressor

* Benchmark system
* RF generally gives great performance for problems like this

In [11]:
xtrain.shape

(14448, 8)

In [12]:
rf=RandomForestRegressor(n_estimators=100).fit(xtrain,ytrain)

In [13]:
# MSE Error
np.mean((rf.predict(xtest)-ytest)**2)

0.27386158637597946

# "Deep" Neural Network

* Two hidden layers
* Investigate the effect of different params

In [17]:
nhidden1=150
nhidden2=100
n_iter=1000
activation=tf.nn.relu
#activation=tf.nn.sigmoid

In [18]:
tf.reset_default_graph()

nx=tf.placeholder(tf.float32,(None,8),name='x')
ny=tf.placeholder(tf.float32,(None,1),name='y')
dropkeep=tf.placeholder(tf.float32)

w1=tf.Variable(tf.truncated_normal((8,nhidden1),stddev=0.01),name='w1')
b1=tf.Variable(tf.zeros(nhidden1),name='b1')
a1=tf.nn.dropout(activation(tf.matmul(nx,w1)+b1),dropkeep,name='a1')

w2=tf.Variable(tf.truncated_normal((nhidden1,nhidden2),stddev=.01),name='w2')
b2=tf.Variable(tf.zeros(nhidden2),name='b2')
a2=tf.nn.dropout(activation(tf.matmul(a1,w2)+b2),dropkeep,name='a2')

wout=tf.Variable(tf.truncated_normal((nhidden2,1),stddev=.01),name='w_out')
bout=tf.Variable(tf.zeros(1),name='b_out')

ypred=tf.matmul(a2,wout)+bout
loss=tf.losses.mean_squared_error(ny,ypred)
#regloss=tf.losses.mean_squared_error(ny,ypred)+.1*tf.nn.l2_loss(w1)+.1*tf.nn.l2_loss(w2)

opt=tf.train.AdamOptimizer().minimize(loss)

In [19]:
try: sess.close()
except: pass

with tf.Session() as sess:
    writer = tf.summary.FileWriter("/home/wlwoon/tmp/day14log", sess.graph)
    sess.run(tf.global_variables_initializer())
    for count in range(n_iter):
        _,tloss=sess.run([opt,loss],feed_dict={dropkeep:.5,nx:xtrain,ny:ytrain.reshape((-1,1))})
        if count % 100 ==0:
            test_loss=sess.run(loss,feed_dict={dropkeep:1.0,nx:xtest,ny:ytest.reshape((-1,1))})
            print("iter #"+str(count)+", training loss is "+str(tloss)+", test loss is "+str(test_loss))
    writer.flush()

iter #0, training loss is 5.57522, test loss is 5.68273
iter #100, training loss is 0.682194, test loss is 0.585924
iter #200, training loss is 0.529074, test loss is 0.448242
iter #300, training loss is 0.519713, test loss is 0.437934
iter #400, training loss is 0.507313, test loss is 0.42968
iter #500, training loss is 0.503863, test loss is 0.42396
iter #600, training loss is 0.492849, test loss is 0.419062
iter #700, training loss is 0.486412, test loss is 0.413798
iter #800, training loss is 0.480265, test loss is 0.409774
iter #900, training loss is 0.474171, test loss is 0.403733


In [19]:
count

199